## Robots.txt

- google: https://www.google.com/robots.txt
- naver: https://www.naver.com/robots.txt

In [1]:
from urllib import robotparser
robot = robotparser.RobotFileParser()
robot.set_url('http://www.naver.com/robots.txt')
robot.read()
robot.can_fetch('Yeti', '/main/imagemontage')

False

In [2]:
from urllib import request

resp = request.urlopen('https://www.google.com')
resp.geturl()
resp.reason
resp.getcode()
print(resp.info())

resp.getheaders()

Date: Thu, 11 Jul 2019 07:46:54 GMT
Expires: -1
Cache-Control: private, max-age=0
Content-Type: text/html; charset=ISO-8859-1
P3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server: gws
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Set-Cookie: 1P_JAR=2019-07-11-07; expires=Sat, 10-Aug-2019 07:46:54 GMT; path=/; domain=.google.com
Set-Cookie: NID=187=RxcYxb4aDv6QvgI9dMxu5GRPH_YfOYrthyjGs5-I6mUIo2I8RulXd47OARPszptt_XIwWlcHjkaJ9RtdjNT9Te2_SgHID51mtkfC58DSyqjL8J79Ps7cIOr-QIz2UGxxW9sjt4filuI5p1sm7ReMQqj-mRh38tf-4cnhZJAWk2g; expires=Fri, 10-Jan-2020 07:46:54 GMT; path=/; domain=.google.com; HttpOnly
Alt-Svc: quic=":443"; ma=2592000; v="46,43,39"
Accept-Ranges: none
Vary: Accept-Encoding
Connection: close




[('Date', 'Thu, 11 Jul 2019 07:46:54 GMT'),
 ('Expires', '-1'),
 ('Cache-Control', 'private, max-age=0'),
 ('Content-Type', 'text/html; charset=ISO-8859-1'),
 ('P3P', 'CP="This is not a P3P policy! See g.co/p3phelp for more info."'),
 ('Server', 'gws'),
 ('X-XSS-Protection', '0'),
 ('X-Frame-Options', 'SAMEORIGIN'),
 ('Set-Cookie',
  '1P_JAR=2019-07-11-07; expires=Sat, 10-Aug-2019 07:46:54 GMT; path=/; domain=.google.com'),
 ('Set-Cookie',
  'NID=187=RxcYxb4aDv6QvgI9dMxu5GRPH_YfOYrthyjGs5-I6mUIo2I8RulXd47OARPszptt_XIwWlcHjkaJ9RtdjNT9Te2_SgHID51mtkfC58DSyqjL8J79Ps7cIOr-QIz2UGxxW9sjt4filuI5p1sm7ReMQqj-mRh38tf-4cnhZJAWk2g; expires=Fri, 10-Jan-2020 07:46:54 GMT; path=/; domain=.google.com; HttpOnly'),
 ('Alt-Svc', 'quic=":443"; ma=2592000; v="46,43,39"'),
 ('Accept-Ranges', 'none'),
 ('Vary', 'Accept-Encoding'),
 ('Connection', 'close')]

## requests package

In [3]:
import requests

In [4]:
def getDownload(url, headers=None, params=None, retries=2):
    resp = requests.get(url, headers=headers, params=params)
    if 500 <= resp.status_code < 600 and retries > 0:
        print(resp.status_code, resp.reason)
        resp = getDownload(url, headers, params, retries-1)
    return resp


def postDownload(url, headers=None, params=None, retries=2):
    resp = requests.post(url, headers=headers, params=params)
    if 500 <= resp.status_code < 600 and retries > 0:
        print(resp.status_code, resp.reason)
        resp = getDownload(url, headers, params, retries-1)
    return resp

In [5]:
resp = getDownload('https://www.google.co.kr')
resp.status_code, resp.reason

(200, 'OK')

In [6]:
resp.headers

{'Date': 'Thu, 11 Jul 2019 07:46:54 GMT', 'Expires': '-1', 'Cache-Control': 'private, max-age=0', 'Content-Type': 'text/html; charset=EUC-KR', 'P3P': 'CP="This is not a P3P policy! See g.co/p3phelp for more info."', 'Content-Encoding': 'gzip', 'Server': 'gws', 'X-XSS-Protection': '0', 'X-Frame-Options': 'SAMEORIGIN', 'Set-Cookie': '1P_JAR=2019-07-11-07; expires=Sat, 10-Aug-2019 07:46:54 GMT; path=/; domain=.google.co.kr, NID=187=SiJCLcZ2WdpIdR2w2FwRM6QGnERQHmf10TyMbbRPc5uhyJZRtzthY2F4O1nbdQf4xjYD3svy-CTQOIafLPzIFwygKxSdZvffslxxypmKGEJMRMzdnRcSpXcsSxjRfHIyf2pUi1hdB6inZTQIziCoOXIOW-r1-5avzwyhqs3c6Zg; expires=Fri, 10-Jan-2020 07:46:54 GMT; path=/; domain=.google.co.kr; HttpOnly', 'Alt-Svc': 'quic=":443"; ma=2592000; v="46,43,39"', 'Transfer-Encoding': 'chunked'}

## Image Crawling

In [7]:
url = 'https://www.google.co.kr/search'
params = {
    'q': '박보영',
    'source': 'lnms',
    'tbm': 'isch',
    'sa': 'X'
}
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

In [8]:
from bs4 import BeautifulSoup

In [9]:
resp = getDownload(url, params=params)
imgUrls = [img['src'] for img in BeautifulSoup(
    resp.text, 'lxml').select('img') if img.has_attr('src')]

In [10]:
for i, imgUrl in enumerate(imgUrls[1:]):
    resp = getDownload(imgUrl)
    ext = '.' + resp.headers['Content-Type'].split('/')[-1]
    with open('이미지/' + str(i) + ext, 'wb') as file:
        file.write(resp.content)

---

## HTTP Status Code

In [11]:
from urllib import request
from urllib import error

try:
    resp = request.urlopen(
        'https://www.google.com/search?q=%ED%8C%8C%EC%9D%B4%EC%8D%AC')
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

403 Forbidden Content-Type: text/html; charset=UTF-8
Date: Thu, 11 Jul 2019 07:46:59 GMT
Server: gws
Cache-Control: private
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Alt-Svc: quic=":443"; ma=2592000; v="46,43,39"
Accept-Ranges: none
Vary: Accept-Encoding
Connection: close




## User Agent

In [12]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
}

In [13]:
from urllib.request import Request

try:
    req = Request(url, headers=headers)
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

In [14]:
resp.code, resp.reason, resp.headers

(200, 'OK', <http.client.HTTPMessage at 0x14f4ac8aac8>)

In [15]:
from urllib.request import Request

#headers = {'User-Agent': None}
try:
    # , headers=headers)
    req = Request(
        'https://www.google.com/search?q=%EB%B0%95%EB%B3%B4%EC%98%81')
    resp = request.urlopen(req)
except error.HTTPError as e:
    print(e.code, e.reason, e.headers)

403 Forbidden Content-Type: text/html; charset=UTF-8
Date: Thu, 11 Jul 2019 07:47:01 GMT
Server: gws
Cache-Control: private
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN
Alt-Svc: quic=":443"; ma=2592000; v="46,43,39"
Accept-Ranges: none
Vary: Accept-Encoding
Connection: close




---

HTTP Method test site: http://httpbin.org

In [16]:
import json

In [17]:
url = 'http://httpbin.org/get'
resp = getDownload(url)
json.loads(resp.text)

{'args': {},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.22.0'},
 'origin': '163.152.3.136, 163.152.3.136',
 'url': 'https://httpbin.org/get'}

In [18]:
resp = getDownload(url, params=params, headers=headers)
json.loads(resp.text)

{'args': {'q': '박보영', 'sa': 'X', 'source': 'lnms', 'tbm': 'isch'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'},
 'origin': '163.152.3.136, 163.152.3.136',
 'url': 'https://httpbin.org/get?q=박보영&source=lnms&tbm=isch&sa=X'}

In [19]:
resp = requests.request('get', url, params=params)
json.loads(resp.text)

{'args': {'q': '박보영', 'sa': 'X', 'source': 'lnms', 'tbm': 'isch'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.22.0'},
 'origin': '163.152.3.136, 163.152.3.136',
 'url': 'https://httpbin.org/get?q=박보영&source=lnms&tbm=isch&sa=X'}

In [20]:
url = 'http://httpbin.org/post'
resp = requests.request('post', url, data=params)
json.loads(resp.text)

{'args': {},
 'data': '',
 'files': {},
 'form': {'q': '박보영', 'sa': 'X', 'source': 'lnms', 'tbm': 'isch'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '55',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.22.0'},
 'json': None,
 'origin': '163.152.3.136, 163.152.3.136',
 'url': 'https://httpbin.org/post'}

In [21]:
def download(method, url, data=None, params=None, headers=None):
    try:
        resp = requests.request(method, url, data=data,
                                params=params, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.code, e.reason)
    finally:
        return resp

In [22]:
def download(method, url, params=None, data=None, headers=None, retries=2):  # 없을수도 있는 것에 None
    try:
        resp = requests.request(
            method, url, params=params, data=data, headers=headers)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        print(e.response.status_code)
        print(e.response.reason)

        if 500 <= e.response.status_code < 600 and retries > 0:
            resp = download(method, url, params=params, data=data,
                            headers=headers, retries=retries-1)
    return resp

In [23]:
headers = None
html = download("get",
                "http://www.crawler-test.com/status_codes/status_403")

403
Forbidden


In [24]:
headers = None
html = download("get",
                "http://www.crawler-test.com/status_codes/status_503")

503
Service Unavailable
503
Service Unavailable
503
Service Unavailable


In [25]:
url = "http://openapi.airkorea.or.kr/openapi/services/rest/ArpltnInforInqireSvc/getCtprvnRltmMesureDnsty"
params = {
    'ServiceKey': 'eZ7xBqg5C3tQDAwJM8kWRO6XWySXi9yEEPI449XNSbb%2BI9AFpA3wUMeeRVqiXhxpc2BOfeyMump%2FlF0pNYCpNA%3D%3D',
    "numOfRows": 10,
    "pageNo": 1,
    "sidoName": "서울",
    "dataTerm": "DAILY",
    "ver": 1.3,
    "_returnType": "json"
}

In [26]:
resp = download('get', url, params=params)
print(resp.text)

<?xml version="1.0" encoding="UTF-8"?>




<response>
	<header>
		<resultCode>30</resultCode>
		<resultMsg>SERVICE KEY IS NOT REGISTERED ERROR.</resultMsg>
	</header>
</response>



In [27]:
import urllib

In [29]:
params['ServiceKey'] = urllib.parse.unquote(params['ServiceKey'])
resp = download('get', url, params=params)
print(resp.text)

{"list":[{"_returnType":"json","coGrade":"1","coValue":"0.4","dataTerm":"","dataTime":"2019-07-11 16:00","khaiGrade":"1","khaiValue":"45","mangName":"도시대기","no2Grade":"1","no2Value":"0.016","numOfRows":"10","o3Grade":"1","o3Value":"0.027","pageNo":"1","pm10Grade":"1","pm10Grade1h":"1","pm10Value":"6","pm10Value24":"5","pm25Grade":"1","pm25Grade1h":"1","pm25Value":"3","pm25Value24":"4","resultCode":"","resultMsg":"","rnum":0,"serviceKey":"","sidoName":"","so2Grade":"1","so2Value":"0.002","stationCode":"","stationName":"중구","totalCount":"","ver":""},{"_returnType":"json","coGrade":"1","coValue":"0.4","dataTerm":"","dataTime":"2019-07-11 16:00","khaiGrade":"1","khaiValue":"35","mangName":"도로변대기","no2Grade":"1","no2Value":"0.021","numOfRows":"10","o3Grade":"1","o3Value":"0.017","pageNo":"1","pm10Grade":"1","pm10Grade1h":"1","pm10Value":"9","pm10Value24":"11","pm25Grade":"1","pm25Grade1h":"1","pm25Value":"5","pm25Value24":"5","resultCode":"","resultMsg":"","rnum":0,"serviceKey":"","sidoName

In [30]:
import json
json.loads(resp.text)

{'list': [{'_returnType': 'json',
   'coGrade': '1',
   'coValue': '0.4',
   'dataTerm': '',
   'dataTime': '2019-07-11 16:00',
   'khaiGrade': '1',
   'khaiValue': '45',
   'mangName': '도시대기',
   'no2Grade': '1',
   'no2Value': '0.016',
   'numOfRows': '10',
   'o3Grade': '1',
   'o3Value': '0.027',
   'pageNo': '1',
   'pm10Grade': '1',
   'pm10Grade1h': '1',
   'pm10Value': '6',
   'pm10Value24': '5',
   'pm25Grade': '1',
   'pm25Grade1h': '1',
   'pm25Value': '3',
   'pm25Value24': '4',
   'resultCode': '',
   'resultMsg': '',
   'rnum': 0,
   'serviceKey': '',
   'sidoName': '',
   'so2Grade': '1',
   'so2Value': '0.002',
   'stationCode': '',
   'stationName': '중구',
   'totalCount': '',
   'ver': ''},
  {'_returnType': 'json',
   'coGrade': '1',
   'coValue': '0.4',
   'dataTerm': '',
   'dataTime': '2019-07-11 16:00',
   'khaiGrade': '1',
   'khaiValue': '35',
   'mangName': '도로변대기',
   'no2Grade': '1',
   'no2Value': '0.021',
   'numOfRows': '10',
   'o3Grade': '1',
   'o3Value

In [31]:
url = 'http://worldagnetwork.com/'
result = requests.get(url)
print(result.content.decode())
result

<html>
<head><title>403 Forbidden</title></head>
<body bgcolor="white">
<center><h1>403 Forbidden</h1></center>
<hr><center>nginx</center>
</body>
</html>



<Response [403]>

In [32]:
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}
result = requests.get(url, headers=headers)
result

<Response [200]>

In [33]:
url = 'https://search.naver.com/search.naver'
params = {'query': '아이린'}
resp = download('get', url, params=params)

In [34]:
# url = 'http://digital.kyobobook.co.kr/digital/ebook/ebookMain.ink'
url = 'http://www.kyobobook.co.kr/search/SearchDigitoryMain.jsp'
data = {'vPstrKeyWord': '아이린', 'vPstrCategory':'DIG'}
data['vPstrKeyWord'] = urllib.parse.unquote(data['vPstrKeyWord'])
resp = download('post', url, data=data)

In [35]:
from html import unescape

In [36]:
url = 'http://www.kyobobook.co.kr/search/SearchCommonMain.jsp'
data = {'vPstrKeyWord': '&#50500;&#51060;&#47536;', 'searchKeyword': '아이린', 'searchCategory': 'TOT'}
resp = download('post', url, data=data)

In [37]:
with open('result.html', 'w') as html:
    html.write(resp.text)

In [38]:
unescape('&#50500;&#51060;&#47536;')

'아이린'

In [39]:
# url = 'http://digital.kyobobook.co.kr/digital/ebook/ebookMain.ink'
url = 'http://www.kyobobook.co.kr/search/SearchDigitoryMain.jsp'
data = {'vPstrKeyWord': 'adsp'}
resp = download('post', url, data=data)

In [40]:
with open('result.html', 'w') as html:
    html.write(resp.text)

Test site: http://pythonscraping.com/pages/cookies/login.html

In [41]:
requests.compat.urlparse('http://pythonscraping.com/pages/cookies/login.html')

ParseResult(scheme='http', netloc='pythonscraping.com', path='/pages/cookies/login.html', params='', query='', fragment='')

In [42]:
url

'http://www.kyobobook.co.kr/search/SearchDigitoryMain.jsp'

In [43]:
url = 'http://pythonscraping.com/pages/cookies/login.html'
print(url)
url = requests.compat.urljoin(url, 'welcome.php')
print(url)
data = {'username': 'Test', 'password': 'password'}
resp = download('post', url, data=data)
print(resp.text)

http://pythonscraping.com/pages/cookies/login.html
http://pythonscraping.com/pages/cookies/welcome.php

<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


---

## Session

### 최초 접속

In [44]:
session = requests.Session()
resp = session.post(url, data=data)
print(resp.text)


<h2>Welcome to the Website!</h2>
Whoops! You logged in wrong. Try again with any username, and the password "password"<br><a href="login.html">Log in here</a>


### 세션 등록 후 접속

In [45]:
resp = session.post(url)
print(resp.text)


<h2>Welcome to the Website!</h2>
You have logged in successfully! <br><a href="profile.php">Check out your profile!</a>


## 쿠키 확인

In [46]:
session.cookies

<RequestsCookieJar[Cookie(version=0, name='loggedin', value='1', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False), Cookie(version=0, name='username', value='Test', port=None, port_specified=False, domain='pythonscraping.com', domain_specified=False, domain_initial_dot=False, path='/pages/cookies', path_specified=False, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={}, rfc2109=False)]>

In [47]:
session = requests.Session()
url = requests.compat.urljoin('https://lms.sunde41.net', '/auth/login')
resp = session.post(url, data={'email': 'tmdals820@naver.com', 'password': '1995-08-20'})

In [49]:
print(resp.text)

<!DOCTYPE html>
<html lang="ko">
<head>
    <title>대시보드 :
        빅데이터 청년인재 고려대학교 과정 학습관리시스템</title>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
    <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, shrink-to-fit=no">
    <meta name="robots" content="noindex,nofollow">
    <link href="/static/vendors/bootstrap.css?v=1.1" rel="stylesheet">
    <link href="/static/vendors/main.css?v=1.1" rel="stylesheet">
    <link href="/static/vendors/icon.css?v=1.0" rel="stylesheet">
    <link href="/static/vendors/preloaders.css?v=1.0" rel="stylesheet">    <link href="/static/vendors/fullcalendar.css?v=1.0" rel="stylesheet">
    <link href="/static/vendors/style.css?v=1.8" rel="stylesheet">
    <link rel="icon" href="/static/favicon/cropped-m_logo-32x32.png" sizes="32x32">
    <link rel="icon" href="/static/favicon/cropped-m_logo-192x192.png" sizes="192x192">
    <link rel="apple-touch-icon-precomposed" href="/static

---

In [51]:
resp = download('get', 'https://www.google.co.kr/search', params={'q': '아이린'})

In [56]:
import re

In [88]:
tags = re.findall('[<][a].*?[>]', resp.text)
result = [tag for tag in tags if re.search('namu.wiki', tag)]
for aTag in result:
    link = re.sub('%25', '%', re.search('href="/url[?]q=(.*)(&amp;.*)(&amp;.*)(&amp;.*)"', aTag).group(1))
    resp = 

https://namu.wiki/w/%EC%95%84%EC%9D%B4%EB%A6%B0(%EB%A0%88%EB%93%9C%EB%B2%A8%EB%B2%B3)
https://namu.wiki/w/%ED%95%99%EB%82%A8%EA%B3%A0%EB%93%B1%ED%95%99%EA%B5%90
https://namu.wiki/w/%EB%B0%B0(%EC%84%B1%EC%94%A8)
https://namu.wiki/w/%EC%B9%A0%EA%B3%A1%EC%A7%80%EA%B5%AC
https://namu.wiki/w/%EB%A0%88%EB%93%9C%EB%B2%A8%EB%B2%B3/%EB%B0%A9%EC%86%A1%20%EB%AA%A9%EB%A1%9D
